In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
from tqdm.notebook import tqdm
import seaborn as sns

# データ保存先
data_folda = '../data/'

* masterからデータを追加

In [5]:

### master data から加えるデータを定義
add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
          'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
          'net_reservation_flg','price_range_lunch_owner','price_range_dinner_owner', 
          'price_range_lunch_user','price_range_dinner_user',
          # 20240312追加
          'all_photo_count','photo_food_count','photo_drink_count',
          'photo_interior_count','photo_exterior_count','official_info_flg','max_visit_month']

# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')

# 直近三カ月の閉店率を計算して保存
# pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-08-31']
# close_ratios=calculate_close_ratio(pseudo_time_series, data_folda)
# close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 三カ月の閉店率を読み込み
close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 2023-01-01からの閉店を予測する。base_date が 2023-01-01の一日前以後が対象
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
pseudo_time_series = add_master_data(pseudo_time_series, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series=pd.merge(pseudo_time_series,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')

pseudo_time_series_test3=pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_master_data(pseudo_time_series_test3, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test3,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')

pseudo_time_series_test6=pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_master_data(pseudo_time_series_test6, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test6,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master.pkl')

In [6]:
pseudo_time_series.columns.values

array(['restaurant_id', 'original_id', 'closed_date', 'base_date',
       'target3', 'target6', 'eval_period', 'id', 'name',
       'prefecture_name', 'head_branch', 'opened_on', 'created_at',
       'genre_first_name', 'genre_second_name', 'genre_third_name',
       'northlatitude', 'eastlongitude', 'net_reservation_flg',
       'price_range_lunch_owner', 'price_range_dinner_owner',
       'price_range_lunch_user', 'price_range_dinner_user',
       'all_photo_count', 'photo_food_count', 'photo_drink_count',
       'photo_interior_count', 'photo_exterior_count',
       'official_info_flg', 'real_opened_date', 'openning_days',
       'days_from_lastvisit', 'close_ratio_genre'], dtype=object)

* V_GOOGLE からデータを追加

In [ ]:
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series = add_v_google(pseudo_time_series, data_folda, add_list)
pseudo_time_series.to_pickle(data_folda+'skeleton_train_with_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_google.pkl')

del pseudo_time_series, pseudo_time_series_test

* V_RETTY からデータを追加

In [ ]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 
       'infos.familiar.users.count', 'infos.familiar.stars',
       'infos.onlineReservation_RETTY', 'infos.access.transferTime1','infos.access.transferTime2','infos.access.transferTime3',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count',
       'infos.familiar.users.genge','infos.reserveEntireStore',"general_rank_RETTY"]
 
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')         
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = add_v_retty(pseudo_time_series, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_retty.pkl')

del pseudo_time_series, pseudo_time_series_test

* HOTPEPPER からデータを追加

In [ ]:
add_list=['restaurant_id','infos.aggregateRating','infos.ratingReview',
          'infos.satisfaction.percentage1','infos.satisfaction.percentage2','infos.satisfaction.percentage3',
          'infos.couponCount','infos.onlineReservation_HOTPEPPER','infos.ownerRegistration','infos.ticket',
          'photos.allCount','photos.appearanceCount','photos.drinkCount','photos.foodCount','photos.otherCount',
          'photos.postCount','menus.courseListCount']


pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hotpepper(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_hotpepper.pkl')

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3= reduce_mem_usage(pseudo_time_series_test3)
pseudo_time_series_test3 = add_v_hotpepper(pseudo_time_series_test3, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_hotpepper.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6= reduce_mem_usage(pseudo_time_series_test6)
pseudo_time_series_test6 = add_v_hotpepper(pseudo_time_series_test6, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_hotpepper.pkl')

del pseudo_time_series, pseudo_time_series_test3, pseudo_time_series_test6

* V_STRUCTURED からデータを取得

In [ ]:
add_list = '"restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount", \
    "aggregateRating.ratingValue", "aggregateRating.reviewCount", \
    "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went", \
    "interactionCount.wanttogo","paymentAccepted"'


pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = add_v_structured(pseudo_time_series, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True)
pseudo_time_series.to_pickle(data_folda+'train_with_structured.pkl')

del pseudo_time_series

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_v_structured(pseudo_time_series_test3, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_structured.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_v_structured(pseudo_time_series_test6, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_structured.pkl')

del pseudo_time_series_test3, pseudo_time_series_test6

* V_TRIPADVISOR から特徴量を追加

In [6]:
def add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False):
    pd_sf = read_table(data_folda,master_name,columns="all",read_snowflake=read_from_snowflake,col="*")

    # general rank を与える
    ranker=rank.Ranker()
    pd_sf=ranker.add_general_rank(pd_sf)
    pd_sf=select_general_rank(pd_sf)
    pd_sf.rename(columns={"general_rank":"general_rank_TRIPADVISOR"},inplace=True)

    pd_sf.rename(columns={"infos.aggregateRating":"infos.aggregateRating_TRIPADVISOR"},inplace=True)
    #add_list=["restaurant_id","infos.aggregateRating_TRIPADVISOR","infos.ratingDetails.Meal","infos.ratingDetails.Service","infos.ratingDetails.Price","infos.ratingDetails.Ambience","infos.reviewCount","infos.qa_count"]


    pd_sf_drop_dum=pd_sf[add_list]

    # oficial_info_flg を categorical に変換
    #pd_sf_drop_dum["infos.official_info_flg"]=pd_sf_drop_dum["infos.official_info_flg"].astype("category")
    pseudo_time_series=pseudo_time_series[["base_date","restaurant_id"]]
    temp=pd.merge(pseudo_time_series,pd_sf_drop_dum,on="restaurant_id",how="left")
    
    return temp

In [7]:
add_list=["restaurant_id","general_rank_TRIPADVISOR",'infos.aggregateRating_TRIPADVISOR',"infos.reviewCount","infos.qa_count"]

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_tripadvisor.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_tripadvisor.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_tripadvisor.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_HITOSARA から特徴量を追加

In [ ]:
# hitosara premium を追加
add_list=["restaurant_id","infos.premium","general_rank_HITOSARA","infos.onlineReservation_HITOSARA"]

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hitosara(pseudo_time_series, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_hitosara.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_hitosara.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_hitosara.pkl')

del pseudo_time_series_test

* v_review_google を追加

In [2]:
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']

# レビューの数
pd_sf=pd.read_csv(data_folda+'feature_aggregation.csv')
# クロール時のこれまでの全レビューのスコアの平均
pd_sf2=pd.read_csv(data_folda+'restaurant_id_sentiment_score.csv')

pd_sf=pd_sf[pd_sf['BASE_DATE']>='2022-12-31']
pd_sf.rename(columns={"BASE_DATE":"base_date","RESTAURANT_ID":"restaurant_id"},inplace=True)
pseudo_time_series=pseudo_time_series[["base_date","restaurant_id"]]
pd_sf["base_date"]=pd.to_datetime(pd_sf["base_date"]).dt.tz_localize(None)
temp=pd.merge(pseudo_time_series,pd_sf,on=["base_date","restaurant_id"],how="left")
temp=pd.merge(temp,pd_sf2,on=["restaurant_id"],how="left")
temp.to_pickle(data_folda+'train_with_feature_aggregation.pkl')

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series=pseudo_time_series[["base_date","restaurant_id"]]
temp=pd.merge(pseudo_time_series,pd_sf,on=["base_date","restaurant_id"],how="left")
temp=pd.merge(temp,pd_sf2,on=["restaurant_id"],how="left")
temp.to_pickle(data_folda+'skeleton_test_3month_feature_aggregation.pkl')

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series=pseudo_time_series[["base_date","restaurant_id"]]
temp=pd.merge(pseudo_time_series,pd_sf,on=["base_date","restaurant_id"],how="left")
temp=pd.merge(temp,pd_sf2,on=["restaurant_id"],how="left")
temp.to_pickle(data_folda+'skeleton_test_6month_feature_aggregation.pkl')

del temp, pseudo_time_series

* 近くの店の閉店率

In [4]:
add_list=['restaurant_id', 'base_date', 'close_ratio_near_restaurant_three_months']
# folda + close_ratio_near_restaurant202*.pkl を全部読み込んで結合
close_ratios=get_close_ratio(data_folda)

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']

pseudo_time_series = pd.merge(pseudo_time_series,close_ratios,on=["restaurant_id","base_date"],how="left")
pseudo_time_series[add_list].to_pickle(data_folda+'skeleton_train_with_near_close_ratio.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test = pd.merge(pseudo_time_series_test,close_ratios,on=["restaurant_id","base_date"],how="left")
pseudo_time_series_test[add_list].to_pickle(data_folda+'skeleton_test_3month_near_close_ratio.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test = pd.merge(pseudo_time_series_test,close_ratios,on=["restaurant_id","base_date"],how="left")
pseudo_time_series_test[add_list].to_pickle(data_folda+'skeleton_test_6month_near_close_ratio.pkl')

del pseudo_time_series, pseudo_time_series_test

../data/close_ratio_near_restaurant20230331.pkl
(875314, 3)
../data/close_ratio_near_restaurant20230630.pkl
(1750628, 3)
../data/close_ratio_near_restaurant20230930.pkl
(2625942, 3)
../data/close_ratio_near_restaurant20230430.pkl
(3501256, 3)
../data/close_ratio_near_restaurant20230731.pkl
(4376570, 3)
../data/close_ratio_near_restaurant20230531.pkl
(5251884, 3)
../data/close_ratio_near_restaurant20230831.pkl
(6127198, 3)
../data/close_ratio_near_restaurant20231031.pkl
(7002512, 3)
../data/close_ratio_near_restaurant20231130.pkl
(7877826, 3)


* 近くの駅の数

In [13]:
# dist_from_station.ipynb で計算した最寄り駅データを読み込んで、concatenate する。
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_train.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = pseudo_time_series[["base_date","restaurant_id"]]
#cat_dummy=pd.read_pickle(data_folda+'category_dummies.pkl').drop(["base_date"],axis=1)
# restaurant_id 以外は、カテゴリカル変数のカラムに _cat_dummy を付与
#cat_dummy.columns = [col+"_cat_dummy" if col!="restaurant_id" else col for col in cat_dummy.columns]

staton=pd.read_csv(data_folda+"/station_address/station20240318free.csv")
station_cd_count=staton.groupby('station_g_cd').size().reset_index(name='count').sort_values('count', ascending=False)
# 同じ station_cd の数をカウントして、付与する
station=pd.merge(staton,station_cd_count,on='station_g_cd',how='left')

# near_station の駅id(station_cd) と距離（dist_from_station）に オリジナルファイル（station） から 乗り換え可能駅数（count） を付与
near_station = pd.read_pickle(data_folda+'train_with_station_dist.pkl')
near_station = near_station[["restaurant_id","station_cd","dist_from_station"]]
near_station=pd.merge(near_station,station[["station_cd","count"]], on='station_cd', how='left')[["restaurant_id","dist_from_station","count"]]

# トレーニングデータに最寄り駅情報を付与
pseudo_time_series = pd.merge(pseudo_time_series,near_station,on="restaurant_id",how="left")
#pseudo_time_series = pd.merge(pseudo_time_series,cat_dummy,on="restaurant_id",how="left")
pseudo_time_series.to_pickle(data_folda+'skeleton_train_near_station.pkl')

# 3ヵ月予測テストデータに最寄り駅情報を付与
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series = pseudo_time_series[["base_date","restaurant_id"]]
pseudo_time_series = pd.merge(pseudo_time_series,near_station,on="restaurant_id",how="left")
pseudo_time_series.to_pickle(data_folda+'skeleton_test_3month_near_station.pkl')

# 6ヵ月予測テストデータに最寄り駅情報を付与
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series = pseudo_time_series[["base_date","restaurant_id"]]
pseudo_time_series = pd.merge(pseudo_time_series,near_station,on="restaurant_id",how="left")
pseudo_time_series.to_pickle(data_folda+'skeleton_test_6month_near_station.pkl')


In [5]:
#train_with_hotpepper.pkl      
#train_with_google.pkl  
#train_with_master.pkl         
#train_with_retty.pkl          
#train_with_tripadvisor.pkl
# 上記五つを pd.concatで結合。 train_with_master 以外は、base_date と restaurant_id が重複するため、あらかじめ削除しておく

concat_list=["train_with_master.pkl","train_with_hotpepper.pkl","train_with_google.pkl","train_with_retty.pkl","train_with_tripadvisor.pkl"
             ,"train_with_hitosara.pkl","train_with_feature_aggregation.pkl","skeleton_train_near_station.pkl","skeleton_train_with_near_close_ratio.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
df=pd.DataFrame()
for i, fn in enumerate(concat_list):
    print(fn)
        
    df_=pd.read_pickle(data_folda+fn)
    if(i==0):
        pass
    else:
        df_=df_.drop(["base_date","restaurant_id"],axis=1)
    
    df=pd.concat([df,df_],axis=1)
    del df_
    
df.to_pickle(data_folda+'train_with_all.pkl')

train_with_master.pkl
train_with_hotpepper.pkl
train_with_google.pkl
train_with_retty.pkl
train_with_tripadvisor.pkl
train_with_hitosara.pkl
train_with_feature_aggregation.pkl
skeleton_train_near_station.pkl
skeleton_train_with_near_close_ratio.pkl


In [6]:
concat_list=["skeleton_test_3month_master.pkl","skeleton_test_3month_hotpepper.pkl","skeleton_test_3month_google.pkl","skeleton_test_3month_retty.pkl","skeleton_test_3month_tripadvisor.pkl"
             ,"skeleton_test_3month_hitosara.pkl","skeleton_test_3month_feature_aggregation.pkl","skeleton_test_3month_near_station.pkl","skeleton_test_3month_near_close_ratio.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
df=pd.DataFrame()
for i, fn in enumerate(concat_list):
    print(fn)
        
    df_=pd.read_pickle(data_folda+fn)
    if(i==0):
        pass
    else:
        df_=df_.drop(["base_date","restaurant_id"],axis=1)
    
    df=pd.concat([df,df_],axis=1)
df.to_pickle(data_folda+'test3_with_all.pkl')

skeleton_test_3month_master.pkl
skeleton_test_3month_hotpepper.pkl
skeleton_test_3month_google.pkl
skeleton_test_3month_retty.pkl
skeleton_test_3month_tripadvisor.pkl
skeleton_test_3month_hitosara.pkl
skeleton_test_3month_feature_aggregation.pkl
skeleton_test_3month_near_station.pkl
skeleton_test_3month_near_close_ratio.pkl


In [7]:
concat_list=["skeleton_test_6month_master.pkl","skeleton_test_6month_hotpepper.pkl","skeleton_test_6month_google.pkl","skeleton_test_6month_retty.pkl","skeleton_test_6month_tripadvisor.pkl"
             ,"skeleton_test_6month_hitosara.pkl","skeleton_test_6month_feature_aggregation.pkl","skeleton_test_6month_near_station.pkl","skeleton_test_6month_near_close_ratio.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
df=pd.DataFrame()
for i, fn in enumerate(concat_list):
    print(fn)
        
    df_=pd.read_pickle(data_folda+fn)
    if(i==0):
        pass
    else:
        df_=df_.drop(["base_date","restaurant_id"],axis=1)
    
    df=pd.concat([df,df_],axis=1)
df.to_pickle(data_folda+'test6_with_all.pkl')

skeleton_test_6month_master.pkl
skeleton_test_6month_hotpepper.pkl
skeleton_test_6month_google.pkl
skeleton_test_6month_retty.pkl
skeleton_test_6month_tripadvisor.pkl
skeleton_test_6month_hitosara.pkl
skeleton_test_6month_feature_aggregation.pkl
skeleton_test_6month_near_station.pkl
skeleton_test_6month_near_close_ratio.pkl


In [ ]:
# pseudo_time_series_filter[general_rank_columns]をプロット
# 横軸を S,A,B,L,F,N,NaNの分布
# 一つのキャンバスにプロット
general_rank_columns=["general_rank_HOTPEPPER","general_rank_GOOGLE","general_rank_HITOSARA","general_rank_RETTY","general_rank_TRIPADVISOR"]
pseudo_time_series_filter=pseudo_time_series[general_rank_columns]
fig, ax = plt.subplots()

for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    ax.hist(pseudo_time_series_filter[column])

plt.show()
"""
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    pseudo_time_series_filter[column].value_counts().plot(kind='bar',title=column,ax=plt.gca())
    plt.show()
"""


In [ ]:
skeleton_train["general_rank_HITOSARA"]

In [ ]:
skeleton_train=pd.read_pickle(data_folda+'skeleton_train.pkl')
skeleton_train=pd.merge(skeleton_train,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_train.to_pickle(data_folda+'skeleton_train_with_general_rank.pkl')

In [ ]:
skeleton_test=pd.read_pickle(data_folda+'skeleton_test.pkl')
skeleton_test=pd.merge(skeleton_test,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_test.to_pickle(data_folda+'skeleton_test_with_general_rank.pkl')

In [ ]:
pseudo_time_series_filter[pseudo_time_series_filter["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]["general_rank_HITOSARA"].value_counts()

In [ ]:
g_filter=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter[g_filter["general_rank_RETTY"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HOTPEPPER"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_TRIPADVISOR"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_STRUCTURED"].isin(["A","B","S","L","N","F"])]

print(g_filter.describe())


In [ ]:
pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_HITOSARA"].value_counts()

In [ ]:
ggoogle=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_GOOGLE"].value_counts()

In [ ]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].dropna().shape[0])
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])

In [ ]:
rank_num.sort_values(by=1,ascending=False)

In [ ]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    num=pseudo_time_series_filter[pseudo_time_series_filter["closed_date"].isna()==False][column].dropna().shape[0]
    ratio=num/len(pseudo_time_series_filter[pseudo_time_series_filter["closed_date"].isna()==False])
    print(column,num,ratio)
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,num,ratio]])
rank_num

In [ ]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    num=pseudo_time_series_filter[column].dropna().shape[0]
    ratio=num/len(pseudo_time_series_filter)
    print(column,num,ratio)
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,num,ratio]])
rank_num

In [ ]:
pseudo_time_series_filter["closed_date"].dropna().shape[0]

In [ ]:
rank_num.sort_values(by=1,ascending=False)

In [ ]:
g_filter=pseudo_time_series_filter
rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]
#num_restaurants=pd.DataFrame()
for column in rank_num.sort_values(by=1,ascending=False)[0].values:
    if(column=="general_rank_STRUCTURED"):
        continue
    g_filter=g_filter[g_filter[column].isin(rlist)]
    num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])

In [ ]:
# general_rank_GOOGLE, general_rank_RETTY, general_rank_HOTPEPPER, general_rank_HITOSARA, general_rank_TRIPADVISORそれぞれがある無しで、どの程度のレストランが抽出されるかを確認

# 各メディアの true False を保存する dataframe を作成
num_restaurants=pd.DataFrame()

#rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]

for is_google in [True,False]:
    for is_retty in [True,False]:
        for is_hotpepper in [True,False]:
            for is_hitosara in [True,False]:
                for is_tripadvisor in [True,False]:
                    g_filter=pseudo_time_series_filter
                    if is_google:
                        g_filter=g_filter[g_filter["general_rank_GOOGLE"].isin(rlist)]
                    if is_retty:
                        g_filter=g_filter[g_filter["general_rank_RETTY"].isin(rlist)]
                    if is_hotpepper:
                        g_filter=g_filter[g_filter["general_rank_HOTPEPPER"].isin(rlist)]
                    if is_hitosara:
                        g_filter=g_filter[g_filter["general_rank_HITOSARA"].isin(rlist)]
                    if is_tripadvisor:
                        g_filter=g_filter[g_filter["general_rank_TRIPADVISOR"].isin(rlist)]
                    print(is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0])
                    
                    num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])

In [ ]:
#num_restaurants　の is_google, is_retty, is_hotpepper, is_hitosara, is_tripadvisor を0,1,2,3,4から代入
num_restaurants.columns=["is_google","is_retty","is_hotpepper","is_hitosara","is_tripadvisor","num_restaurants"]

In [ ]:
num_restaurants

In [ ]:
#num_restaurants=num_restaurants.drop([0,1,2,3,4,5],axis=1)
num_restaurants["num_restaurants"]=num_restaurants["num_restaurants"].astype(int)
num_restaurants.sort_values(by="num_restaurants",ascending=False,inplace=True)
num_restaurants[["is_google","is_retty","is_hotpepper","is_hitosara","is_tripadvisor","num_restaurants"]]
num_restaurants
